# Exercise 1: SQL

In this notebook I provide my answer for the SQL exercise. (Please scroll to the end since the problem definition changed. The solution is at the end of this file.)

In [2]:
dataset_url = 'https://figshare.com/ndownloader/files/35249488'

In [3]:
import numpy
import os
import seaborn as sns
import pandas
import plotly_express as px
import plotly.graph_objects as go

In [4]:
%%time

# - getting the dataset
cache_path = os.path.abspath('../../resources/raw_data/dataset.pkl')
if os.path.isfile(cache_path):
    df = pandas.read_pickle(cache_path)
else:
    df = pandas.read_csv(dataset_url).drop(columns=['Unnamed: 0'])
    df.to_pickle(cache_path)

CPU times: user 0 ns, sys: 18.7 ms, total: 18.7 ms
Wall time: 18 ms


In [5]:
df.head()

,VL,CD4,Rel CD4,Gender,Ethnic,Base Drug Combo,Comp. INI,Comp. NNRTI,Extra PI,Extra pk-En,VL (M),CD4 (M),Drug (M),PatientID,Timepoints
0,1141.895800,1070.03560,32.654160,1.0,4.0,0.0,3.0,1.0,5.0,0.0,1.0,1.0,0.0,0,0
1,134.190550,444.54190,14.775723,1.0,4.0,0.0,3.0,3.0,5.0,0.0,1.0,1.0,1.0,0,1
2,47.274055,230.53404,15.087534,1.0,4.0,0.0,3.0,3.0,5.0,0.0,0.0,0.0,1.0,0,2
3,120.055940,419.28403,26.615877,1.0,4.0,1.0,3.0,3.0,5.0,0.0,1.0,1.0,1.0,0,3
4,27.249084,230.72127,13.609289,1.0,4.0,1.0,3.0,3.0,5.0,0.0,1.0,1.0,1.0,0,4


In [6]:
from pandasql import sqldf
from sqlalchemy import text
pysqldf = lambda q: sqldf(q, globals())

In [7]:
df_agg = pysqldf(
"""
SELECT df1.PatientID, df1.Timepoints, df1.Gender, df1.CD4, df2.CD4,
df1.CD4 - COALESCE(df2.CD4, df1.CD4) as lagged_CD4
FROM df as df1 LEFT JOIN df as df2 ON df1.PatientID = df2.PatientID AND df1.Timepoints = df2.Timepoints + 1;

""")
df_agg

,PatientID,Timepoints,Gender,CD4,CD4,lagged_CD4
0,0,0,1.0,1070.03560,NaN,0.00000
1,0,1,1.0,444.54190,1070.03560,-625.49370
2,0,2,1.0,230.53404,444.54190,-214.00786
3,0,3,1.0,419.28403,230.53404,188.74999
4,0,4,1.0,230.72127,419.28403,-188.56276
...,...,...,...,...,...,...
534955,8915,55,1.0,815.18480,247.44243,567.74237
534956,8915,56,1.0,1441.06570,815.18480,625.88090
534957,8915,57,1.0,424.17233,1441.06570,-1016.89337
534958,8915,58,1.0,295.58610,424.17233,-128.58623


In [36]:
df_agg = pysqldf(
"""
SELECT AVG(df2.lagged_CD4) AS avg_lagged_CD4, df2.Timepoints, df2.Gender
FROM
(
    SELECT df1.PatientID, df1.Timepoints, df1.Gender, df1.CD4, df2.CD4,
    df1.CD4 - COALESCE(df2.CD4, df1.CD4) as lagged_CD4
    FROM df as df1 LEFT JOIN df as df2 ON df1.PatientID = df2.PatientID AND df1.Timepoints = df2.Timepoints + 1
) AS df2 GROUP BY df2.Gender, df2.Timepoints;

""")
df_agg

,avg_lagged_CD4,Timepoints,Gender
0,0.000000,0,1.0
1,-121.405805,1,1.0
2,69.365054,2,1.0
3,-47.204000,3,1.0
4,1.721680,4,1.0
...,...,...,...
115,-297.262163,55,2.0
116,-143.309887,56,2.0
117,89.723519,57,2.0
118,189.386613,58,2.0


In [38]:
df_agg = pysqldf(
"""
SELECT df.*, df3.avg_lagged_CD4
FROM df LEFT JOIN  (
    SELECT AVG(df2.lagged_CD4) AS avg_lagged_CD4, df2.Timepoints, df2.Gender
    FROM
    (
        SELECT df1.PatientID, df1.Timepoints, df1.Gender, df1.CD4, df2.CD4,
        df1.CD4 - COALESCE(df2.CD4, df1.CD4) as lagged_CD4
        FROM df as df1 LEFT JOIN df as df2 ON df1.PatientID = df2.PatientID AND df1.Timepoints = df2.Timepoints + 1
    ) AS df2 GROUP BY df2.Gender, df2.Timepoints
) AS df3 ON df.Gender = df3.Gender AND df.Timepoints = df3.Timepoints;
""")
df_agg

,VL,CD4,Rel CD4,Gender,Ethnic,Base Drug Combo,Comp. INI,Comp. NNRTI,Extra PI,Extra pk-En,VL (M),CD4 (M),Drug (M),PatientID,Timepoints,avg_lagged_CD4
0,1141.895800,1070.03560,32.654160,1.0,4.0,0.0,3.0,1.0,5.0,0.0,1.0,1.0,0.0,0,0,0.000000
1,134.190550,444.54190,14.775723,1.0,4.0,0.0,3.0,3.0,5.0,0.0,1.0,1.0,1.0,0,1,-121.405805
2,47.274055,230.53404,15.087534,1.0,4.0,0.0,3.0,3.0,5.0,0.0,0.0,0.0,1.0,0,2,69.365054
3,120.055940,419.28403,26.615877,1.0,4.0,1.0,3.0,3.0,5.0,0.0,1.0,1.0,1.0,0,3,-47.204000
4,27.249084,230.72127,13.609289,1.0,4.0,1.0,3.0,3.0,5.0,0.0,1.0,1.0,1.0,0,4,1.721680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534955,22.383543,815.18480,33.672344,1.0,4.0,0.0,3.0,2.0,5.0,0.0,1.0,0.0,1.0,8915,55,64.570597
534956,43.911175,1441.06570,28.304905,1.0,4.0,0.0,3.0,2.0,5.0,0.0,0.0,0.0,1.0,8915,56,-144.882779
534957,23.014017,424.17233,34.530067,1.0,4.0,0.0,3.0,2.0,5.0,0.0,0.0,0.0,1.0,8915,57,-29.332491
534958,31.049942,295.58610,24.711374,1.0,4.0,0.0,3.0,2.0,5.0,0.0,0.0,0.0,1.0,8915,58,-98.646958


Alright, the exercise seems to be modified:


```
Hello Shayan,

One clarification on the SQL exercise - there should be one row per unique gender/timestamp pair, not the same number of rows as in the original dataset. I update the language in the exercise accordingly.

Thanks,
```

In [9]:
df_agg = pysqldf(
"""
SELECT df1.PatientID, df1.Timepoints, df1.Gender, df1.CD4,
df1.CD4 - COALESCE(df2.CD4, df1.CD4) as lagged_CD4
FROM df as df1 LEFT JOIN df as df2 ON df1.PatientID = df2.PatientID AND df1.Timepoints = df2.Timepoints + 1;
""")
df_agg

,PatientID,Timepoints,Gender,CD4,lagged_CD4
0,0,0,1.0,1070.03560,0.00000
1,0,1,1.0,444.54190,-625.49370
2,0,2,1.0,230.53404,-214.00786
3,0,3,1.0,419.28403,188.74999
4,0,4,1.0,230.72127,-188.56276
...,...,...,...,...,...
534955,8915,55,1.0,815.18480,567.74237
534956,8915,56,1.0,1441.06570,625.88090
534957,8915,57,1.0,424.17233,-1016.89337
534958,8915,58,1.0,295.58610,-128.58623


In [15]:
df_agg = pysqldf(
"""
SELECT 
    df3.PatientID, df3.Timepoints, df3.Gender, df3.lagged_CD4,
    AVG(df3.lagged_CD4) OVER (
    PARTITION BY df3.PatientID ORDER BY df3.Timepoints ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS svn_avg_lagged_CD4
    
FROM (
    SELECT df1.PatientID, df1.Timepoints, df1.Gender,
    df1.CD4 - COALESCE(df2.CD4, df1.CD4) as lagged_CD4
    FROM df as df1 LEFT JOIN df as df2 ON df1.PatientID = df2.PatientID AND df1.Timepoints = df2.Timepoints + 1
) AS df3;
""")
df_agg

,PatientID,Timepoints,Gender,lagged_CD4,svn_avg_lagged_CD4
0,0,0,1.0,0.00000,0.000000
1,0,1,1.0,-625.49370,-312.746850
2,0,2,1.0,-214.00786,-279.833853
3,0,3,1.0,188.74999,-162.687892
4,0,4,1.0,-188.56276,-167.862866
...,...,...,...,...,...
534955,8915,55,1.0,567.74237,66.624771
534956,8915,56,1.0,625.88090,121.306214
534957,8915,57,1.0,-1016.89337,-91.363210
534958,8915,58,1.0,-128.58623,-14.087249


In [1]:
(0 - 625.49 - 214.) / 3.

-279.83

In [18]:
df_agg = pysqldf(
"""
SELECT df4.Gender, df4.Timepoints, AVG(svn_avg_lagged_CD4)
FROM (
    SELECT df3.PatientID, df3.Timepoints, df3.Gender, df3.lagged_CD4,
    AVG(df3.lagged_CD4) OVER (
    PARTITION BY df3.PatientID ORDER BY df3.Timepoints ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS svn_avg_lagged_CD4
    
    FROM (
        SELECT df1.PatientID, df1.Timepoints, df1.Gender,
        df1.CD4 - COALESCE(df2.CD4, df1.CD4) as lagged_CD4
        FROM df as df1 LEFT JOIN df as df2 ON df1.PatientID = df2.PatientID AND df1.Timepoints = df2.Timepoints + 1
    ) AS df3
) AS df4 GROUP BY df4.Gender, df4.Timepoints LIMIT 10;
""")
df_agg

,Gender,Timepoints,AVG(svn_avg_lagged_CD4)
0,1.0,0,0.000000
1,1.0,1,-60.702903
2,1.0,2,-17.346917
3,1.0,3,-24.811188
4,1.0,4,-19.504614
5,1.0,5,6.297980
6,1.0,6,37.026534
7,1.0,7,1.753741
8,1.0,8,36.437426
9,1.0,9,2.061530
